In [126]:
import os
import cv2
import glob
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [127]:
def get_masks(train_images_folder, train_masks_folder):
    mask_files = glob.glob(os.path.join(train_masks_folder, '*.tif'))

    # Preprocess mask filenames
    mask_dict = {}
    for m_path in mask_files:
        base_name = os.path.basename(m_path).split('.')[0].rsplit('_', 2)[0].rsplit('_occluded', 1)[0]
        mask_dict.setdefault(base_name, []).append(m_path)

    images, masks = [], []
    for img_path in glob.glob(os.path.join(train_images_folder, '*.png')):
        img_name = os.path.basename(img_path).split('.')[0]

        if img_name in mask_dict and len(mask_dict[img_name]) == 4:
            images.append(img_path)
            masks.append(mask_dict[img_name])

    return images, masks

In [128]:
def find_petri_dish_thresholding(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Find contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours based on area to find the Petri dish contour
    petri_dish_contour = None
    min_contour_area = 4000 

    for contour in contours:
        if cv2.contourArea(contour) > min_contour_area:
            petri_dish_contour = contour
            break

    # Extract bounding box coordinates of the Petri dish
    x, y, w, h = cv2.boundingRect(petri_dish_contour)

    # Ensure the bounding box is a square
    side_length = max(w, h)
    x_center, y_center = x + w // 2, y + h // 2

    # Crop the Petri dish from the original image
    petri_dish = image[y:y + h, x:x + w]

    # Resize the cropped Petri dish to 2770x2770
    petri_dish = cv2.resize(petri_dish, (2770, 2770))

    # Convert the cropped Petri dish to grayscale
    petri_dish_gray = cv2.cvtColor(petri_dish, cv2.COLOR_BGR2GRAY)

    return petri_dish_gray

In [129]:
def padder(image, patch_size):
    h = image.shape[0]
    w = image.shape[1]
    height_padding = ((h // patch_size) + 1) * patch_size - h
    width_padding = ((w // patch_size) + 1) * patch_size - w

    top_padding = int(height_padding/2)
    bottom_padding = height_padding - top_padding

    left_padding = int(width_padding/2)
    right_padding = width_padding - left_padding

    padded_image = cv2.copyMakeBorder(image, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=[0, 0, 0])

    return padded_image

In [130]:
def preprocess_data(images, masks, patch_size):
    patch_images, patch_masks = [], []

    for img_path, mask_paths in zip(images, masks):
        # Load image
        image = cv2.imread(img_path)

        # Check if masks exist
        if not all(os.path.exists(mask_path) for mask_path in mask_paths):
            continue

        # Find and crop Petri dish
        petri_dish = find_petri_dish_thresholding(image)

        # Padding
        padded_petri_dish = padder(petri_dish, patch_size)

        # Patching
        patch_size = 256
        num_channels = 3  # Assuming RGB images

        # Calculate the number of patches in each dimension
        num_patches_h = padded_petri_dish.shape[0] // patch_size
        num_patches_w = padded_petri_dish.shape[1] // patch_size

        # Initialize an empty list for patches
        patch_images = []

        # Extract patches directly using numpy slicing
        for i in range(num_patches_h):
            for j in range(num_patches_w):
                if padded_petri_dish.ndim == 3:  # Color image
                    patch = padded_petri_dish[i * patch_size:(i + 1) * patch_size, j * patch_size:(j + 1) * patch_size, :]
                else:  # Grayscale image
                    patch = padded_petri_dish[i * patch_size:(i + 1) * patch_size, j * patch_size:(j + 1) * patch_size]
                    patch = np.expand_dims(patch, axis=-1)  # Add the third dimension for compatibility with color images
                patch_images.append(patch)

        # Convert to numpy array
        patch_images = np.array(patch_images)


        # Append to lists
        patch_masks.extend([cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE) for mask_path in mask_paths])

    return patch_images, np.array(patch_masks)

In [131]:
# Define your dataset path
dataset_folder = r"C:\Users\User\Desktop\2023-24b-fai2-adsai-yuliiabobrovytska226038\1. Computer Vision\DataLab tasks\Task 4\dataset"
train_images_folder = os.path.join(dataset_folder, 'train_images', 'train')
train_masks_folder = os.path.join(dataset_folder, 'train_masks', 'train')

In [132]:
# Set patch size
patch_size = 256

In [133]:
# Get images and masks
images, masks = get_masks(train_images_folder, train_masks_folder)

In [134]:
# Preprocess data
patch_images, patch_masks = preprocess_data(images, masks, patch_size)

In [135]:
patch_images.shape

(121, 256, 256, 1)

In [136]:
patch_masks.shape

(352, 3006, 4202)

In [137]:
# Assuming num_classes is defined earlier in your code
num_classes = 5

In [138]:
def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = TP / (Positives+K.epsilon())
        return recall
    
    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = TP / (Pred_Positives+K.epsilon())
        return precision
    
    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [139]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2])
        total = K.sum(K.square(y_true), axis=[1, 2]) + K.sum(K.square(y_pred), axis=[1, 2])
        union = total - intersection
        return (intersection + K.epsilon()) / (union + K.epsilon())

    return K.mean(f(y_true, y_pred), axis=-1)

In [140]:
def simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
# Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    s = inputs

    # Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    # Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

     
    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1, iou])
    model.summary()
    
    return model

In [141]:
# Set the image dimensions
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = patch_size, patch_size, 3

In [142]:
# Build the model
model = simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_38 (Conv2D)             (None, 256, 256, 16  448         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_18 (Dropout)           (None, 256, 256, 16  0           ['conv2d_38[0][0]']              
                                )                                                           

In [143]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(patch_images, patch_masks, test_size=0.2, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [121, 352]

In [144]:
# Set up callbacks for training
checkpoint = ModelCheckpoint("model_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True, mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=16, callbacks=[checkpoint, early_stopping, reduce_lr])

In [ ]:
# Evaluate the model on the test set
results = model.evaluate(X_test, y_test, batch_size=16)

In [ ]:
# Print the test set loss, accuracy, F1 score, and IoU
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])
print("Test F1 Score:", results[2])
print("Test IoU:", results[3])

In [ ]:
# Plot learning curves
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()